In [1]:
import glob

import numpy
import torch
import PIL.Image
import torchvision.tv_tensors
import torchvision.transforms.v2

transforms = torchvision.transforms.v2.Compose([
    torchvision.transforms.v2.CenterCrop((224, 224)),
    torchvision.transforms.v2.Resize((64, 64)),
    torchvision.transforms.v2.ToDtype(torch.float32, scale = True)
])
def transforms_wrapper(image, mask):
    image = torchvision.tv_tensors.Image(image)
    mask = torchvision.tv_tensors.Mask(mask)
    return transforms(image, mask)

def load_dataset(path):
    images, masks = [], []
    for filename in glob.glob(f"{path}/image/*"):
        image, mask = transforms_wrapper(
            PIL.Image.open(filename),
            PIL.Image.open(filename.replace("image", "mask").replace("jpg", "png"))
        )
        images.append(image.permute((1, 2, 0)).numpy())
        masks.append(mask.squeeze().numpy())
    return numpy.array(images)[:75, :, :, 0], numpy.array(masks)[:75]

train_images, train_masks = load_dataset("midv500p/train")
test_images, test_masks = load_dataset("midv500p/val")

In [2]:
import cvtda.topology
extractor = cvtda.topology.FeatureExtractor(
    n_jobs = 1,
    binarizer_thresholds = [ 0.4 ],
    num_radial_filtrations = 2
)
train_features = extractor.fit_transform(train_images, "seg_train")
test_features = extractor.transform(test_images, "seg_test")

Calculating inverted images
GreyscaleExtractor: processing seg_train/greyscale, do_fit = True
Got the result from ./seg_train/greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
DiagramVectorizer: fitting complete
Got the result from ./seg_train/greyscale/features.npy
GreyscaleExtractor: processing seg_train/greyscale, do_fit = False
Got the result from ./seg_train/greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
Got the result from ./seg_train/greyscale/features.npy
GreyscaleExtractor: processing seg_train/inverted_greyscale, do_fit = True
Got the result from ./seg_train/inverted_greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
DiagramVectorizer: fitting complete
Got the result from ./seg_train/inverted_greyscale/features.npy
GreyscaleExtractor: processing seg_train/inverted_greyscale, do_fit = False
Got the result from ./seg_train/inverted_greyscale/diagrams.npy
Applying Scaler to persistence diagrams.
Got the result from ./seg_train/inverted

In [4]:
import cvtda.segmentation
cvtda.segmentation.segment(
    train_images, train_features, train_masks,
    test_images, test_features, test_masks,
    n_epochs = 3
)

Trying no - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x16e8e75e0>
Parameters: 29187817


100%|██████████| 3/3 [00:08<00:00,  2.89s/it, loss=2.57, lr=1e-5, Accuracy=0.562, AUC-ROC=0.645, Precision=0, Recall=0, F1-score=0, Jaccard score=0]  


{'model': 'Без признаков', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.644880888268072, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}
Trying images - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x16e8e7520>
Parameters: 29187817


100%|██████████| 3/3 [00:01<00:00,  2.37it/s, loss=2.57, lr=1e-5, Accuracy=0.562, AUC-ROC=0.646, Precision=0, Recall=0, F1-score=0, Jaccard score=0] 


{'model': 'Без топологических признаков', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.6458871142228453, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}
Trying topological - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x16e8e74f0>
Parameters: 29187817


100%|██████████| 3/3 [00:03<00:00,  1.07s/it, loss=2.57, lr=1e-5, Accuracy=0.562, AUC-ROC=0.646, Precision=0, Recall=0, F1-score=0, Jaccard score=0] 


{'model': 'Только топологические признаки', 'Accuracy': 0.5620865360383065, 'AUC-ROC': 0.6462318908953529, 'Precision': 0.0, 'Recall': 0.0, 'F1-score': 0.0, 'Jaccard score': 0.0}
Trying combined - <cvtda.segmentation.MiniUnet.MiniUnet object at 0x16e8e74c0>
Parameters: 29261545


100%|██████████| 3/3 [00:01<00:00,  1.94it/s, loss=2.5, lr=1e-5, Accuracy=0.438, AUC-ROC=0.666, Precision=0.438, Recall=1, F1-score=0.609, Jaccard score=0.438]   


{'model': 'Комбинированные признаки', 'Accuracy': 0.43791346396169356, 'AUC-ROC': 0.6662780025330377, 'Precision': 0.43791346396169356, 'Recall': 1.0, 'F1-score': 0.6090957139453557, 'Jaccard score': 0.43791346396169356}


,model,Accuracy,AUC-ROC,Precision,Recall,F1-score,Jaccard score
0,Без признаков,0.562087,0.644881,0.000000,0.0,0.000000,0.000000
1,Без топологических признаков,0.562087,0.645887,0.000000,0.0,0.000000,0.000000
2,Только топологические признаки,0.562087,0.646232,0.000000,0.0,0.000000,0.000000
3,Комбинированные признаки,0.437913,0.666278,0.437913,1.0,0.609096,0.437913
